In [1]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random


In [2]:
#Instalar gecko
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service


#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox(service=Service(executable_path=GeckoDriverManager().install()))

SessionNotCreatedException: Message: Failed to start browser /snap/firefox/current/firefox.launcher: IO Error: No such file or directory (os error 2)


In [ ]:
URL="https://technical.city/en/video/rating"

In [ ]:

def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    idLoader=kargs.get("idLoader",query)
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,idLoader,by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_element(kargs.get("by",By.XPATH),query)
    waitFinishLoad(driver,idLoader,by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def clickEvent(driver,element,**kargs):
    """Click en un elemento que funciona con evento

    Args:
        driver (WebDriver): WebDriver
        element (Element): Elemento que tiene el evento
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    if kargs.get("normal",False):
        element.click
    else:
        try:
            driver.execute_script(element.get_dom_attribute("onClick"))
        except:
            element.click
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))

def elementsBy(query,driver,**kargs):
    """Devuelve una lista de elements dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        List<Element>: Lista de elementos
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_elements(kargs.get("byq",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element
# endregion

In [ ]:
driver.get(URL)

In [ ]:
elementBy("//*[@role='dialog']/div[2]/div/button[2]",driver,by=By.XPATH,needsWait=False).click()

In [ ]:
# id-footer
from io import StringIO
from bs4 import BeautifulSoup
import pandas as pd


df = pd.DataFrame([])
pastElements=[]
laps=0
while True:
    elements=elementsBy("//tr",driver,by=By.XPATH)
    driver.execute_script("arguments[0].scrollIntoView();", elements[-1])
    if pastElements==elements:
        laps+=1
    else:
        laps=0
    if laps==100:
        break
    pastElements=elements


soup=BeautifulSoup(driver.page_source,'html.parser')
df=pd.read_html(StringIO(str(soup.findAll("table")[0])))[0]
#df=df[~df.iloc[:,0].str.startswith('Página')]

In [ ]:
df = df[['Graphics card','Type','Performance','Architecture','Year','Price now','TDP']]

In [ ]:
df

,Graphics card,Type,Performance,Architecture,Year,Price now,TDP
0,GeForce RTX 4090,desktop,100.00,Ada Lovelace,2022,1756 USD,450 W
1,GeForce RTX 4080,desktop,89.39,Ada Lovelace,2022,1451 USD,320 W
2,GeForce RTX 4080 SUPER,desktop,88.54,Ada Lovelace,2024,−,320 W
3,GeForce RTX 4070 Ti SUPER,desktop,82.14,Ada Lovelace,2024,−,285 W
4,GeForce RTX 4070 Ti,desktop,81.99,Ada Lovelace,2023,837 USD,285 W
...,...,...,...,...,...,...,...
1446,GeForce2 Ultra,desktop,−,Celsius,2000,−,−
1447,GeForce4 Ti 4400,desktop,−,Kelvin,2002,−,−
1448,GeForce4 Ti 4800 SE,desktop,−,Kelvin,2003,−,−
1449,Quadro NVS 280 PCI,workstation,−,Rankine,2003,100 USD,13 W


In [ ]:
df=df[df.isna().sum(axis=1)<3]

In [ ]:
len(df)

1447

In [ ]:
import os
import pathlib


df.to_csv(os.path.join(str(pathlib.Path().resolve()),"GPU.csv"))